In [0]:
#Tratamiento de datos
import pandas as pd
import numpy as np

#Graficos
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)
import sys
# Establecer la opción de impresión para mostrar el array completo
np.set_printoptions(threshold=sys.maxsize)

## 1. Modelo de Machine Learning Con Datos Balanceados en Series Temporales

 Leemos los datos de PRESENTATION la tabla Delta con los datos Crudos Sin Balanceo

In [0]:
#1. Leemos los datos de PROCEESED la tabla Delta 
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/tablacaracteristicas_congestion_tabladelta")
datos = df_delta.toPandas()
datos.head()

In [0]:
#datos.shape

In [0]:
datos['congestion'].value_counts()

#### 2.Eliminar las variables que no ingresaran al Modelo de ML

In [0]:
#datos.dtypes

In [0]:
# 2. Limpieza de variables No prioritarias
columnas_a_eliminar = ['nombre_equipo','nombre','start_time_alert','end_time_alert','Event_Date']

# 2.1 Filtrar las columnas que existen en el DataFrame
columnas_existentes = [col for col in columnas_a_eliminar if col in datos_sin_balanceo.columns]

# 2.2 Verificar si hay columnas para eliminar
if columnas_existentes:
    datos_sin_balanceo.drop(columnas_existentes, axis=1, inplace=True)

In [0]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

# Convertimos a index la fecha
datos = datos.set_index('instant_date_t')

# Filtrar las fechas donde congestion == 1
fechas_congestion_1 = datos[datos['congestion'] == 1].index

In [0]:
fechas_congestion_1

In [0]:
# Generar un rango de fechas completo dentro del rango del DataFrame
fecha_min = datos.index.min()
fecha_max = datos.index.max()
rango_completo = pd.date_range(start=fecha_min, end=fecha_max, freq='S')

In [0]:
fecha_min, fecha_max

In [0]:
rango_completo

In [0]:
# Excluir las fechas de congestion == 1 del rango completo
fechas_disponibles = rango_completo.difference(fechas_congestion_1)

In [0]:
fechas_disponibles

In [0]:
# Aplicar SMOTE para generar nuevas muestras de la clase minoritaria (congestion == 0)
smote = SMOTE(sampling_strategy='auto', random_state=42) # la clase minoritaria será sobre-muestreada hasta sea igual al mayoritario
X = np.arange(len(datos)).reshape(-1, 1)  # Crear una secuencia numérica, SMOTE espera que X sea una matriz de dos dimensiones
y = datos['congestion']  #variable objetivo (target)

In [0]:
# Aplicar SMOTE
X_res, y_res = smote.fit_resample(X, y)

# Convertir las nuevas muestras a DataFrame
datos_resampleados = pd.DataFrame({'index': X_res.flatten(), 'congestion': y_res})

In [0]:
datos_resampleados.shape

In [0]:
datos_resampleados['congestion'].value_counts()

In [0]:
# Filtrar las nuevas muestras generadas por SMOTE (la diferencia entre las originales y las nuevas)
nuevas_muestras = datos_resampleados[len(datos):]

In [0]:
nuevas_muestras.head()

In [0]:
# Asignar nuevas fechas secuenciales de fechas_disponibles a las nuevas muestras
nuevas_fechas_asignadas = fechas_disponibles[:len(nuevas_muestras)]
nuevas_muestras['instant_date'] = nuevas_fechas_asignadas

In [0]:
nuevas_fechas_asignadas

In [0]:
nuevas_muestras.head()

In [0]:
#Solo se han de agregar las etiquetas que tienen la clase minoritaria en este caso 0
nuevas_muestras['congestion'].value_counts()

In [0]:
# Crear el DataFrame final combinando los datos originales y las nuevas muestras
nuevas_muestras.set_index('instant_date', inplace=True)
datos_balanceados = pd.concat([datos, nuevas_muestras[['congestion']]]).sort_index()

In [0]:
datos.dtypes

#### 3. Hacer el Balanceo de datos mediante SMOTE (aumentando la clase minoritoria)

In [0]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

# Convertimos a index la fecha
datos = datos.set_index('instant_date_t')

# Filtrar las fechas donde congestion == 1
fechas_congestion_1 = datos[datos['congestion'] == 1].index

# Generar un rango de fechas completo dentro del rango del DataFrame
fecha_min = datos.index.min()
fecha_max = datos.index.max()
rango_completo = pd.date_range(start=fecha_min, end=fecha_max, freq='S')

# Excluir las fechas de congestion == 1 del rango completo
fechas_disponibles = rango_completo.difference(fechas_congestion_1)

# Aplicar SMOTE para generar nuevas muestras de la clase minoritaria (congestion == 0)
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Usar todas las columnas como características para SMOTE excepto 'congestion'
X = datos.drop(columns=['congestion'])
y = datos['congestion']

# Convertir las fechas del índice a números para usarlas con SMOTE
X['instant_date_num'] = X.index.astype(int) / 10**9  # Convertir a segundos

# Aplicar SMOTE
X_res, y_res = smote.fit_resample(X, y)

# Convertir de nuevo a DataFrame
X_res = pd.DataFrame(X_res, columns=X.columns)
y_res = pd.Series(y_res, name='congestion')

# Filtrar las nuevas muestras generadas por SMOTE (la diferencia entre las originales y las nuevas)
nuevas_muestras = X_res[len(datos):]
nuevas_congestion = y_res[len(datos):]

# Asignar nuevas fechas secuenciales de fechas_disponibles a las nuevas muestras
nuevas_fechas_asignadas = fechas_disponibles[:len(nuevas_muestras)]
nuevas_muestras['instant_date'] = nuevas_fechas_asignadas

# Crear el DataFrame final combinando los datos originales y las nuevas muestras
nuevas_muestras = nuevas_muestras.set_index('instant_date')
nuevas_muestras = nuevas_muestras.drop(columns=['instant_date_num'])
nuevas_muestras['congestion'] = nuevas_congestion.values

# Combinar los datos originales y las nuevas muestras
datos_balanceados = pd.concat([datos, nuevas_muestras]).sort_index()

In [0]:
datos_balanceados.head()

In [0]:
#### Verificamos si se hizo el Balanceo adecuadamente

In [0]:
print(datos_balanceados['congestion'].value_counts())

#### 3.1 Verificamos la peridocidad entre nuestra serie temporal, si hay periocidad exacta, o es por distintas periocidades

In [0]:
#2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos_balanceados.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos_balanceados.index.to_series().diff().dt.total_seconds()

# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
diferencias_frecuencias = df_time_diffs.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
print(diferencias_frecuencias)

In [0]:
spark.sql("SHOW DATABASES").show(truncate=False)

In [0]:
from pyspark.sql.functions import col

# Resetear el índice de Pandas y renombrarlo como "fecha_hora"
datos_balanceados.reset_index(inplace=True)
datos_balanceados.rename(columns={"index": "instant_date_t"}, inplace=True)

# Convertir el DataFrame de Pandas a un DataFrame de Spark
spark_datos = spark.createDataFrame(datos_balanceados)

# Nombre de la tabla Delta a guardar
nombre_tabla_delta = "dbproyectocongestion_presentation.data_congestion_serietemp_balanceada"

# Verificar si ya existe la tabla Delta
if spark.catalog.tableExists(nombre_tabla_delta):
    # Eliminar la tabla Delta existente
    spark.sql("DROP TABLE IF EXISTS " + nombre_tabla_delta)

# 12. Guardar los datos preprocesados en una tabla Delta
spark_datos.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(nombre_tabla_delta)

#### 3.2 Uniformizamos la periocidad de la serie temporal, a la deseada

Leemos los datos de PRESENTATION la tabla Delta con los datos Balanceados

In [0]:
#1. Leemos los datos de PROCEESED la tabla Delta 
df_delta2 = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/data_congestion_serietemp_balanceada")
datos_cong_balanceado = df_delta2.toPandas()
datos_cong_balanceado.head()

In [0]:
print(datos_cong_balanceado['congestion'].value_counts())

In [0]:
datos_cong_balanceado.shape

In [0]:
# #2. Ajuste de los datos a Series Temporales
# #2.1 Se establece la columna 'Time' como el índice del DataFrame "datos"
datos_cong_balanceado = datos_cong_balanceado.set_index('instant_date_t')

# #2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos_cong_balanceado.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos_cong_balanceado.index.to_series().diff().dt.total_seconds()

# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
# diferencias_frecuencias = df_time_diffs.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
# print(diferencias_frecuencias)

#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
datos_cong_balanceado = datos_cong_balanceado[df_time_diffs != 0]

#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
datos_cong_balanceado = datos_cong_balanceado.asfreq(freq='5T', method='bfill')

datos_cong_balanceado.head()

#### 3.3 Verificamos los en cuantos datos nos quedamos luego de uniformizar la periocidad de la serie temporal

In [0]:
datos_cong_balanceado['congestion'].value_counts()

In [0]:
df_time_diffs2 = datos_cong_balanceado.index.to_series().diff().dt.total_seconds()

# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
diferencias_frecuencias2 = df_time_diffs2.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
print(diferencias_frecuencias2)

In [0]:
# Verificar si hay valores nulos en el DataFrame
valores_nulos = datos_cong_balanceado.isnull().sum()
valores_nulos

In [0]:
datos_cong_balanceado.head()

### 4. Modelos de Machine learning en Problemas de Series Temporales

#### 4.1 Modelo Forecaster LGBM(Light Gradient Boosting Machine)

 4.1.1 Separacion de los datos en Train-Validation-Test

In [0]:
datos_cong_balanceado.index.min(),datos_cong_balanceado.index.max()

In [0]:
#Separacion de los datos en Train-Validation-Test
datos_cong_balanceado = datos_cong_balanceado.loc['2024-03-27 01:15:28' : '2024-04-23 23:59:30']
fin_train = '2024-04-15 23:59:30'
fin_validacion = '2024-04-19 23:59:30'
datos_train = datos_cong_balanceado.loc[:fin_train, :] #20 dias
datos_val = datos_cong_balanceado.loc[fin_train:fin_validacion, :] #5 dias
datos_test = datos_cong_balanceado.loc[fin_validacion: , :] # 5 dias

print(f"Fechas train: {datos_train.index.min()} ... {datos_train.index.max()} (n={len(datos_train)})")
print(f"Fechas validacion: {datos_val.index.min()} ... {datos_val.index.max()} (n={len(datos_val)})")
print(f"Fechas test: {datos_test.index.min()} ... {datos_test.index.max()} (n={len(datos_test)})")

In [0]:
print(datos_train.shape,datos_val.shape, datos_test.shape)

4.1.2 Exploracion Grafica

In [0]:
#Grafico de la serie temporal

#crea una nueva figura y un par de ejes (axes) dentro de la figura
fig, ax = plt.subplots(figsize=(15,4)) # figsize=(15,4) establece el tamaño de la figura en 15 unidades de ancho y 4 unidades de alto.

#Las series se colorean automáticamente según una paleta de colores predefinida por matplotlib
datos_train.congestion.plot(ax=ax, label="entrenamiento", linewidth=1)
datos_val.congestion.plot(ax=ax, label="validacion", linewidth=1)
datos_test.congestion.plot(ax=ax, label="test", linewidth=1)
ax.set_title('Congestion ')
ax.legend();

4.1.3 Grafico de Autocorrelacion 

In [0]:
#Grafico autocorrelación

#Crea una figura y un conjunto de ejes (ax)
fig, ax = plt.subplots(figsize=(10,4))

#Aquí utilizamos la función plot_acf para trazar la función de autocorrelación
#ax=ax indica que el gráfico se dibujará en los ejes que has creado anteriormente. 
# lags=72 define el número máximo de intervalos de tiempo pasados que se considerarán al calcular la función de autocorrelación. 
plot_acf(datos_cong_balanceado.congestion, ax=ax, lags=800)
plt.show()

#Podemos ver que la correlacion de la prediccion esta altamente correlaciona con el la demanda en el dia anterior a la misma hora.

In [0]:
#Grafico de Autocorrelacion parcial
fig, ax = plt.subplots(figsize=(10,4))

#plot_pacf para trazar la función de autocorrelación parcial
plot_pacf(datos_cong_balanceado.congestion, ax=ax, lags=60)

# PACF muestra la correlación directa entre la serie y sus 
# valores anteriores hasta 60 intervalos de tiempo atrás, teniendo en cuenta las correlaciones indirecta
plt.show()

#### 4.1.4 Algoritmo de LGBM(Ligth Bosting)

In [0]:
# Modelado y Forescasting
from lightgbm import LGBMRegressor, LGBMClassifier    #Regressor mas potente de gradient bossting
from skforecast.ForecasterAutoreg import ForecasterAutoreg  # Modelo autoregressivo
from skforecast.model_selection import grid_search_forecaster # Encontrar los mejores hiperparametros(optimizar el modelo)
from skforecast.model_selection import backtesting_forecaster # Evaluar el modelo si hubiese estado en produccion

In [0]:
#Crear  y entrenar forescaster
forescaster = ForecasterAutoreg(
    regressor=LGBMClassifier(max_depth=3, learning_rate=0.1, n_estimators=70),
    lags= 5 #[1, 2, 3, 24]
)
forescaster #forescaster.summary()

Entrenamiento del Forescaster

In [0]:
forescaster.fit(y=datos_cong_balanceado.loc[:fin_validacion, 'congestion']) #Entrenamiento con conjuntos de train + validacion
forescaster

Predicciones

In [0]:
predicciones = forescaster.predict(steps=1000)
predicciones

In [0]:
forescaster.last_window

In [0]:
# Obtener las probabilidades para las predicciones
probabilidades = forescaster.regressor.predict_proba(forescaster.last_window.values.reshape(1, -1))

In [0]:
probabilidades

In [0]:
# Inicializar una lista para almacenar las probabilidades para cada predicción
probabilidades_predicciones = []

# Iterar sobre las predicciones y obtener las probabilidades para cada una
for pred in predicciones:
    # Obtener las probabilidades para la predicción actual
    proba = forescaster.regressor.predict_proba(forescaster.last_window.values.reshape(1, -1))
    probabilidades_predicciones.append(proba)

In [0]:
probabilidades_predicciones

#### 4.1.5 Algoritmo Time Series Forest Clasification 

In [0]:
#!pip install sktime

In [0]:
#pip install --upgrade pip

In [0]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [0]:
datos_cong_balanceado.tail()

Seleccionamos la variable target y las variables predictoras

In [0]:
# Reformatear los datos en un formato compatible con sktime
X = datos_cong_balanceado[['x', 'y']].values.reshape(-1, 1, 2)

# Tomar las etiquetas objetivo (y) como una serie temporal
y = datos_cong_balanceado['congestion']

In [0]:
#Ultimos registros del numpy array
X[-2:]

In [0]:
#Primeros registros del numpy array
X[:5]

In [0]:
y[:5]

#### Entrenamiento del Modelo ML

In [0]:
fechas = datos_cong_balanceado.index
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test, fechas_train, fechas_test = train_test_split(X, y, fechas, test_size=0.2, shuffle=False)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False) # shuffle=False, los datos no se mezclarán y se dividirán #secuencialmente según el orden en el que aparecen en tus datos (adecuado para problemas de series temporales)
classifier = TimeSeriesForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
y_pred_probability = classifier.predict_proba(X_test)

In [0]:
fechas_train[:5]

In [0]:
X_train[-2:]

In [0]:
y_train[:5]

In [0]:
y_test[:5]

#### Solo nos quedamos con la columna de la derecha, Donde seria (Probabilidad de que suceda la "Clase 1" en este caso SI HAY CONGESTION)

In [0]:
import numpy as np
import sys

# Suponiendo que y_pred_probability es tu matriz de probabilidades
y_pred_columna_derecha = y_pred_probability[:, 1]

# Establecer la opción de impresión para mostrar el array completo
np.set_printoptions(threshold=sys.maxsize)

# Imprimir el array completo de probabilidades
print(y_pred_columna_derecha)

In [0]:
accuracy_score(y_test, y_pred)

In [0]:
import matplotlib.pyplot as plt

# Crea la figura con el tamaño especificado
plt.figure(figsize=(15, 6))

# Graficar las predicciones
plt.plot(fechas_test, y_test, linestyle='-', marker='o', color='blue', label='Real')
plt.plot(fechas_test, y_pred_columna_derecha, linestyle='--', marker='x', color='red', label='Predicción')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.title('Comparación entre Real y Predicción')
plt.legend()
plt.show()


In [0]:
import matplotlib.pyplot as plt

# Crea la figura con el tamaño especificado
plt.figure(figsize=(15, 6))

# Graficar las predicciones
plt.plot(fechas_test, y_test, linestyle='-', marker='o', color='blue', label='Real')
plt.plot(fechas_test, y_pred, linestyle='--', marker='x', color='red', label='Predicción')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.title('Comparación entre Real y Predicción')
plt.legend()
plt.show()

In [0]:
datos_cong_balanceado.tail()

In [0]:
window_size = 5

# Seleccionar solo las variables específicas que deseas incluir en X_future
variables_seleccionadas = ['x', 'y']

# Obtener las últimas observaciones de las variables seleccionadas
X_future = datos_cong_balanceado.tail(window_size)[variables_seleccionadas].values

X_future

In [0]:
len(X_future)

In [0]:
X_future[2:2+window_size]

In [0]:
X_window2 = X_future[4:4+window_size]
X_window2

In [0]:
X_window2.reshape(1, -1)

In [0]:
datos_cong_balanceado.tail()

In [0]:
# Obtener los últimos cinco pasos anteriores de las dos variables para el pronóstico
window_size = 5
X_future = datos_cong_balanceado.tail(window_size).values

# Realizar el pronóstico para cada paso de tiempo futuro
y_pred_future = []

for i in range(len(X_future)):
    # Obtener las últimas cinco observaciones de las dos variables
    X_window = X_future[i:i+window_size]
    
    # Realizar la predicción de congestión con el clasificador entrenado
    congestion_pred = classifier.predict_proba(X_window.reshape(1, -1))
    
    # Agregar la predicción a la lista de pronósticos futuros
    y_pred_future.append(congestion_pred)

# Convertir la lista de pronósticos futuros en un arreglo numpy
y_pred_future = np.array(y_pred_future)
# Suponiendo que y_pred_probability es tu matriz de probabilidades
y_pred_future = y_pred_future[:, :, 1]
# Visualizar las predicciones futuras
print("Predicciones futuras de congestión:", y_pred_future)

In [0]:
len(X_future)

In [0]:
# Preparar una lista para almacenar las predicciones futuras
y_pred_future2 = []

# Definir el tamaño de la ventana de predicción futura
window_size2 = 1  # Por ejemplo, predecir los próximos 5 pasos

variables_seleccionadas = ['x', 'y']

# Obtener las últimas observaciones de las variables seleccionadas
X_future = datos_cong_balanceado.tail(window_size)[variables_seleccionadas].values

#X_future = datos_cong_balanceado.tail(window_size).values

# Iterar a través de tus datos futuros
for i in range(len(X_future) - window_size2 + 1):
    # Obtener la ventana de datos actual
    X_window = X_future[i:i+window_size]
    
    # Realizar la predicción de congestión con tu modelo clasificador
    congestion_pred = classifier.predict_proba(X_window.reshape(1, -1))  # Ajustar la forma según tu modelo
    
    # Tomar solo la última predicción y agregarla a la lista
    y_pred_future2.append(congestion_pred[-1])

# Convertir la lista de predicciones en un arreglo numpy
y_pred_future2 = np.array(y_pred_future2)
# Suponiendo que y_pred_probability es tu matriz de probabilidades
y_pred_future2 = y_pred_future2[:, 1]

# Visualizar las predicciones
print("Predicciones futuras:", y_pred_future2)

In [0]:
datos_cong_balanceado.tail(10).values

In [0]:
datos_cong_balanceado.tail(2)[variables_seleccionadas].values

In [0]:
variables_seleccionadas = ['x', 'y','z']

# Obtener las últimas observaciones de las variables seleccionadas
X_future_single_step = datos_cong_balanceado.tail(1)[variables_seleccionadas].values

# Realizar la predicción para un solo paso hacia adelante
congestion_pred_single_step = classifier.predict_proba(X_future_single_step)


# Visualizar la predicción
print("Probabilidad de congestión para el próximo paso:", congestion_pred_single_step)

In [0]:
X_future_single_step

XGB

In [0]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Supongamos que tienes tus datos de series temporales en X_time_series y tus variables exógenas en X_exogenous
# Supongamos que tienes tus etiquetas en y

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_ts, X_test_ts, X_train_exog, X_test_exog, y_train, y_test = train_test_split(X_time_series, X_exogenous, y, test_size=0.2, shuffle=False)

# Crear una matriz DMatrix para XGBoost
train_dmatrix = xgb.DMatrix(data=X_train_ts, label=y_train)
test_dmatrix = xgb.DMatrix(data=X_test_ts)

# Definir los parámetros del modelo
params = {
    'objective': 'binary:logistic',  # para problemas de clasificación binaria
}

# Entrenar el modelo XGBoost
model = xgb.train(params=params, dtrain=train_dmatrix)

# Realizar predicciones de probabilidades de clase
y_pred_probs = model.predict(test_dmatrix)

print("Predicciones de probabilidades de clase:", y_pred_probs)


### Algoritmo XGB Time Series

In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
datos_cong_balanceado.head()

In [0]:
FEATURES = ['x','y','bearing_t','speed_t']

x = datos_cong_balanceado[FEATURES]
y = datos_cong_balanceado['congestion']

In [0]:

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False, random_state=1)
#shuffle=False, los datos no se mezclarán y se dividirán #secuencialmente según el orden(ideal para time series)

# Generamos los datasets de train y test
train  = pd.concat([X_train,y_train], axis=1)
test  = pd.concat([X_test,y_test], axis=1)

In [0]:
X_train.head()

In [0]:
xgb_model = xgb.XGBClassifier(n_estimators=300,max_depth=5,objective='binary:logistic',reg_alpha=0.1) #reg_alpha>>regulazrizacion Lasso educir la complejidad del modelo y evitar #el sobreajuste
#eval_set>>monitorear el rendimiento del modelo en los datos de evaluación durante el #entrenamiento
#objective='binary:logistic'> Para problemas de clasificacion binaria

xgb_model.fit(X_train,y_train,eval_set=[(X_train,y_train), (X_test,y_test)],eval_metric=['auc','error'],
              early_stopping_rounds=50,verbose=True)
#verbose=True >> Mostrar en pantalla las iteraciones
#'auc'>>AUC es una métrica que evalúa la capacidad del modelo para distinguir entre clases positivas y negativas
# 'error'>> cuanto se equivoca el modelo en predecir (100-error)
# logloss>>un logloss más bajo indica un mejor rendimiento del modelo.

Feature Importance

In [0]:
f1 = pd.DataFrame(data=xgb_model.feature_importances_,index=xgb_model.feature_names_in_,columns=['importantes/variables'])

f1.sort_values('importantes/variables').plot(kind='barh',title='Variables')
plt.show()

In [0]:
import sys
y_pred_prob = xgb_model.predict_proba(X_test)
y_pred_prob = np.round(y_pred_prob,3)
y_pred_prob = y_pred_prob[:, 1]  # Filtramos columna derecha, para dejar probabilidad que suceda CLASE 1(Conges)
y_pred = xgb_model.predict(X_test)

# Establecer la opción de impresión para mostrar el array completo
np.set_printoptions(threshold=sys.maxsize)
y_pred_prob

In [0]:
y_pred

Armamos nuetros datos para graficar las serie temporal original vs prediccion en el periodo de datos de Test

In [0]:
test["prediction-prob"] = y_pred_prob  #y_pred en dimension 1d
test["prediction"] = y_pred  #y_pred en dimension 1d
datos_cong_balanceado = datos_cong_balanceado.merge(test[['prediction-prob','prediction']], how='left', left_index=True,right_index=True) #left_index=True, right_index=True > la fusión se realizará basándose índices de las filas
datos_cong_balanceado.tail()

In [0]:
ax = datos_cong_balanceado[['congestion']].plot(figsize=(15,5)) #crea un gráfico de la columna 'congestion_x' del DF

# Se utiliza el ax=ax especifica que esta serie debe agregarse al mismo gráfico que se creó anteriormente
datos_cong_balanceado['prediction-prob'].plot(ax=ax, style='.') #style='.'
plt.legend(['Original','Prediction'])
ax.set_title('Time Series XGB')
plt.show()

In [0]:
test.index.max()

In [0]:
ax = datos_cong_balanceado.loc[(datos_cong_balanceado.index > '2024-04-18 09:50:28') & (datos_cong_balanceado.index < '2024-04-22 09:50:28')]['congestion'].plot(figsize=(15,5), title='Data Semanal')
datos_cong_balanceado.loc[(datos_cong_balanceado.index > '2024-04-18 09:50:28') & (datos_cong_balanceado.index < '2024-04-22 09:50:28')]['prediction-prob'].plot(style='.')
plt.legend(['Original','Prediction-prob'])
plt.show()

Medimos el Performance del Modelo (En este caso con Accuracy, por ser un modelo de Clasificacion)

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [0]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn import metrics
names = ['sin_congestion','congestion']  #Aqui se especifica los nombres de las clases >  En orden desde 0>Sin congestion  1>Congestion, 2>etc, etc
matriz_confusion = confusion_matrix(y_test, y_pred)  #Poner el orden> Clase real , clase predicha
print(metrics.classification_report(y_test,y_pred, digits=4))
plot_confusion_matrix(conf_mat=matriz_confusion, figsize =(4,4), class_names=names, show_normed=False)
plt.tight_layout()

#### Outlier Removal of Time Series

In [0]:
datos_cong_balanceado['congestion'].plot(style='.',
                        figsize=(15,5),                       
                        title='Congestion Time Serie')                 
plt.show()

Analisis de Outliers (Histograma)

In [0]:
datos_cong_balanceado['congestion'].plot(kind='hist', bins=300)

1. Time Series Croos Validation

In [0]:
datos_cong_balanceado.head()

In [0]:
from sklearn.model_selection import TimeSeriesSplit

#Se construye el Generador del CROSS VALIDATION
tss = TimeSeriesSplit(n_splits=5, test_size=24*1, gap=24)
#24*365*1 : por ejemplo quieres predecir 1 año

#ordenamos la secuencia para un problema de serie temporal
datos_cong_balanceado = datos_cong_balanceado.sort_index() 

In [0]:
# Generamos los subgraficos para los tramos del Croos Validation, en este caso (Hemos partido en 5 partes)
fig, axs = plt.subplots(5, 1, figsize=(15,15), sharex=True)

#Inicializamos el contador, para movernos en los tramas
fold = 0
#Aplicamos el generador de CROSS VALIDATION
for train_idx , val_idx in tss.split(datos_cong_balanceado):
    train2 = datos_cong_balanceado.iloc[train_idx]
    test2 = datos_cong_balanceado.iloc[val_idx]
    train2['congestion'].plot(ax=axs[fold],
                                label='Train Set',
                                title=f'Data Train/Test Split Fold {fold}')
    test2['congestion'].plot(ax=axs[fold],
                                label='Test Set')
    axs[fold].axvline(test2.index.min(), color='black', ls='--')
    fold += 1
plt.show()

In [0]:
val_idx

2. Forecasting Horizon Explaind
- El horizonte de pronóstico es la longitud de tiempo en el futuro para la cual se preparan los pronósticos. Estos generalmente varían desde horizontes de pronóstico a corto plazo (menos de tres meses) hasta a largo plazo (más de dos años).

In [0]:
datos_cong_balanceado[['x','y','bearing_t','speed_t']].head()

3. Lag Features (Funciones de Retraso)
- What was the target (x) days in the past (¿Cuál era el objetivo (x) días en el pasado?")

In [0]:
datos_cong_balanceado['congestion'].to_dict()

In [0]:
target_map = datos_cong_balanceado['congestion'].to_dict()

In [0]:
datos_cong_balanceado.index.min(), datos_cong_balanceado.index.max(), 

In [0]:
def add_lags(df):
    #Hacemos un diccionario con los valores del TARGET, para completar luego
    target_map = datos_cong_balanceado['congestion'].to_dict()

    #datos_cong_balanceado['lag1'] = (datos_cong_balanceado.index - pd.Timedelta('1 days')).map(target_map)  #Retroceso 1, en este caso (1 dia)
    datos_cong_balanceado['lag1'] = (datos_cong_balanceado.index - pd.Timedelta('5T')).map(target_map)  #Retroceso 1, en este caso (1 dia)
    datos_cong_balanceado['lag2'] = (datos_cong_balanceado.index - pd.Timedelta('10T')).map(target_map)  #Retroceso 2, en este caso (2 dia)
    datos_cong_balanceado['lag3'] = (datos_cong_balanceado.index - pd.Timedelta('15T')).map(target_map)  #Retroceso 3, en este caso (3 dia)
    return df

In [0]:
# Eliminar las columnas 'columna1' y 'columna2'
#datos_cong_balanceado.drop(['lag1', 'lag2'], axis=1, inplace=True)

# axis=1 especifica que las columnas deben ser eliminadas
# inplace=True modifica el DataFrame original en su lugar

In [0]:
datos_cong_balanceado = add_lags(datos_cong_balanceado)
datos_cong_balanceado.head()

Entrenar el Modelo Usando Cross Validation

In [0]:
def create_features(df):
    #Create time series features based on time series index
    df = df.copy()
    df['hora'] = df.index.hour
    df['minuto'] = df.index.min
    return df

In [0]:
from sklearn.model_selection import TimeSeriesSplit

#Se construye el Generador del CROSS VALIDATION
tss = TimeSeriesSplit(n_splits=5, test_size=24*1, gap=24)
#24*365*1 : por ejemplo quieres predecir 1 año

#ordenamos la secuencia para un problema de serie temporal
datos_cong_balanceado = datos_cong_balanceado.sort_index() 

#Inicializamos el contador, para movernos en los tramas
fold = 0
preds = []
scores = []

#Aplicamos el generador de CROSS VALIDATION
for train_idx , val_idx in tss.split(datos_cong_balanceado):
    train2 = datos_cong_balanceado.iloc[train_idx]
    test2 = datos_cong_balanceado.iloc[val_idx]
    
    train2 = create_features(train2)
    test2 = create_features(test2)

    FEATURES = ['x','y','bearing_t','speed_t','lag1', 'lag2', 'lag3']
    TARGET = 'congestion'

    X_train2 = train2[FEATURES]
    y_train2 = train2[TARGET]

    X_test2 = test2[FEATURES]
    y_test2 = test2[TARGET]

    xgb_model = xgb.XGBClassifier(n_estimators=300,max_depth=5,objective='binary:logistic',reg_alpha=0.1) #reg_alpha>>regulazrizacion Lasso educir la complejidad del modelo y evitar #el sobreajuste
    #eval_set>>monitorear el rendimiento del modelo en los datos de evaluación durante el #entrenamiento
    #objective='binary:logistic'> Para problemas de clasificacion binaria

    xgb_model.fit(X_train2,y_train2,eval_set=[(X_train2,y_train2), (X_test2,y_test2)],eval_metric=['auc','error'],
                early_stopping_rounds=50,verbose=True)

    y_pred2 = xgb_model.predict(X_test2)
    preds.append(y_pred2)
    score = np.sqrt(mean_squared_error(y_test2,y_pred2))
    scores.append(score)

In [0]:
print(f'Score acroos folds {np.mean(scores):0.4f}')
print(f'Fold scores: {scores}')

Entrenar el Modelo sin CV

In [0]:
datos_cong_balanceado.head()

In [0]:
#‘W’, or ‘D’
#‘days’, or ‘day’
#‘hours’, ‘hour’, ‘hr’, or ‘h’
#‘minutes’, ‘minute’, ‘min’, or ‘m’
#‘seconds’, ‘second’, ‘sec’, or ‘s’
def add_lags(df):
    #Hacemos un diccionario con los valores del TARGET, para completar luego
    target_map = df['congestion'].to_dict()

    #datos_cong_balanceado['lag1'] = (datos_cong_balanceado.index - pd.Timedelta('1 days')).map(target_map)  #Retroceso 1, en este caso (1 dia)
    #df['lag1'] = (df.index - pd.Timedelta('5T')).map(target_map)  #Retroceso 1, en este caso (5 minutos)
    df['lag1'] = (df.index - pd.Timedelta(25, 'hours')).map(target_map)  #Retroceso 1, en este caso (1 dia)
    df['lag2'] = (df.index - pd.Timedelta(32, 'hours')).map(target_map)  #Retroceso 2, en este caso (2 dia)
    df['lag3'] = (df.index - pd.Timedelta(40, 'hours')).map(target_map)  #Retroceso 3, en este caso (3 dia)
    return df

In [0]:
datos_cong_balanceado = add_lags(datos_cong_balanceado)

FEATURES = ['x','y','bearing_t','speed_t','lag1', 'lag2', 'lag3','hora','minuto']
TARGET = 'congestion'

x = datos_cong_balanceado[FEATURES]
y = datos_cong_balanceado['congestion']

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False, random_state=1)

# Generamos los datasets de train y test
train  = pd.concat([X_train,y_train], axis=1)
test  = pd.concat([X_test,y_test], axis=1)

xgb_model = xgb.XGBClassifier(n_estimators=300,max_depth=5,objective='binary:logistic',reg_alpha=0.1) #reg_alpha>>regulazrizacion Lasso educir la complejidad del modelo y evitar #el sobreajuste
    #eval_set>>monitorear el rendimiento del modelo en los datos de evaluación durante el #entrenamiento
    #objective='binary:logistic'> Para problemas de clasificacion binaria

xgb_model.fit(X_train,y_train,eval_set=[(X_train,y_train), (X_test,y_test)],eval_metric=['auc','error'],
                early_stopping_rounds=50,verbose=True)

Feature Importance

In [0]:
f1 = pd.DataFrame(data=xgb_model.feature_importances_,index=xgb_model.feature_names_in_,columns=['importantes/variables'])

f1.sort_values('importantes/variables').plot(kind='barh',title='Variables')
plt.show()

In [0]:
import sys
y_pred_prob = xgb_model.predict_proba(X_test)
y_pred_prob = np.round(y_pred_prob,3)
y_pred_prob = y_pred_prob[:, 1]  # Filtramos columna derecha, para dejar probabilidad que suceda CLASE 1(Conges)
y_pred = xgb_model.predict(X_test)

# Establecer la opción de impresión para mostrar el array completo
np.set_printoptions(threshold=sys.maxsize)
y_pred_prob

In [0]:
test.head()

Armamos nuetros datos para graficar las serie temporal original vs prediccion en el periodo de datos de Test

In [0]:
test["prediction-prob"] = y_pred_prob  #y_pred en dimension 1d
test["prediction"] = y_pred  #y_pred en dimension 1d
datos_cong_balanceado = datos_cong_balanceado.merge(test[['prediction-prob','prediction']], how='left', left_index=True,right_index=True) #left_index=True, right_index=True > la fusión se realizará basándose índices de las filas
datos_cong_balanceado.tail()

In [0]:
ax = datos_cong_balanceado[['congestion']].plot(figsize=(15,5)) #crea un gráfico de la columna 'congestion_x' del DF

# Se utiliza el ax=ax especifica que esta serie debe agregarse al mismo gráfico que se creó anteriormente
datos_cong_balanceado['prediction-prob'].plot(ax=ax, style='.') #style='.'
plt.legend(['Original','Prediction'])
ax.set_title('Time Series XGB')
plt.show()

In [0]:
test.index.max()

In [0]:
ax = datos_cong_balanceado.loc[(datos_cong_balanceado.index > '2024-04-18 09:50:28') & (datos_cong_balanceado.index < '2024-04-22 09:50:28')]['congestion'].plot(figsize=(15,5), title='Data Semanal')
datos_cong_balanceado.loc[(datos_cong_balanceado.index > '2024-04-18 09:50:28') & (datos_cong_balanceado.index < '2024-04-22 09:50:28')]['prediction-prob'].plot(style='.')
plt.legend(['Original','Prediction-prob'])
plt.show()

Medimos el Performance del Modelo (En este caso con Accuracy, por ser un modelo de Clasificacion)

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [0]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
names = ['sin_congestion','congestion']  #Aqui se especifica los nombres de las clases >  En orden desde 0>Sin congestion  1>Congestion, 2>etc, etc
matriz_confusion = confusion_matrix(y_test, y_pred)  #Poner el orden> Clase real , clase predicha
print(metrics.classification_report(y_test,y_pred, digits=4))
plot_confusion_matrix(conf_mat=matriz_confusion, figsize =(4,4), class_names=names, show_normed=False)
plt.tight_layout()

4. Predicting the Future
- Reentrenamiento en todos los datos (Entrenar el Modelo con los Lags, si aun no lo has entrenado asi)
- Para predecir el futuro, necesitamos un dataframe vacío para rangos de fechas futuras
- Ejecutar esas fechas a través de nuestro código de creación de características + creación de rezagos"

In [0]:
datos_cong_balanceado.index.max() + pd.Timedelta(minutes=5)

In [0]:
datos_cong_balanceado.index.max() + pd.Timedelta(days=1)

In [0]:
futuretesting=pd.date_range(datos_cong_balanceado.index.max() + pd.Timedelta(minutes=5), datos_cong_balanceado.index.max() + pd.Timedelta(days=1) , freq='5T')

In [0]:
futuretesting[0]

In [0]:
futuretesting[0] - pd.Timedelta(days=1)

In [0]:
futuretesting - pd.Timedelta(days=1)

In [0]:
#Creature future dataframe

# #1. Creamos el esqueleto del df del Pronostico (con las fecha Inicio y Fin del Pronostico (en este caso 1 dia en el futuro en rango de 5minutos))
future=pd.date_range(datos_cong_balanceado.index.max() + pd.Timedelta(minutes=5), datos_cong_balanceado.index.max() + pd.Timedelta(days=1) , freq='5T')
future_df=pd.DataFrame(index=future)

# #Se agrega la columna de FUTURE en el df pronostico com TRUE (para indicar que son el futuro)
future_df['isFuture'] = True

# #Se agrega la columna de FUTURE en el df Original con FALSE (para indicar no es futuro) 
datos_cong_balanceado['isFuture'] = False

#Se hace una concatenacion para agregar las filas de future_df al df_original
df_and_future = pd.concat([datos_cong_balanceado, future_df])

In [0]:
df_and_future.head()

In [0]:
variables_pronostico = ['congestion','x','y','bearing_t','speed_t','lag1', 'lag2', 'lag3','hora','minuto','isFuture']
df_and_future = df_and_future[variables_pronostico]
df_and_future.tail(30)

In [0]:
df_and_future.head()

In [0]:
def create_features(df):
    #Create time series features based on time series index
    df = df.copy()
    df['hora'] = df.index.hour
    df['minuto'] = df.index.minute
    return df

In [0]:
datos_cong_balanceado.shape

In [0]:
df_and_future.shape

In [0]:
df_and_future.index.max() 

In [0]:
#‘W’, or ‘D’
#‘days’, or ‘day’
#‘hours’, ‘hour’, ‘hr’, or ‘h’
#‘minutes’, ‘minute’, ‘min’, or ‘m’
#‘seconds’, ‘second’, ‘sec’, or ‘s’
def add_lags(df):
    #Hacemos un diccionario con los valores del TARGET, para completar luego
    target_map = df['congestion'].to_dict()

    #datos_cong_balanceado['lag1'] = (datos_cong_balanceado.index - pd.Timedelta('1 days')).map(target_map)  #Retroceso 1, en este caso (1 dia)
    #df['lag1'] = (df.index - pd.Timedelta('5T')).map(target_map)  #Retroceso 1, en este caso (5 minutos)
    df['lag1'] = (df.index - pd.Timedelta(25, 'hours')).map(target_map)  #Retroceso 1, en este caso (1 dia)
    df['lag2'] = (df.index - pd.Timedelta(32, 'hours')).map(target_map)  #Retroceso 2, en este caso (2 dia)
    df['lag3'] = (df.index - pd.Timedelta(40, 'hours')).map(target_map)  #Retroceso 3, en este caso (3 dia)
    return df

In [0]:
# df_and_future = create_features(df_and_future)
df_and_future = add_lags(df_and_future)

In [0]:
# df_and_future.tail(310)

In [0]:
# df_and_future.tail(50)

In [0]:
#Filtramos solo el Dataset Esqueleto para el Pronostico
future_dates_pronostico = df_and_future.query('isFuture').copy()
future_dates_pronostico.head()

In [0]:
# df_and_future['x'].loc[future - pd.Timedelta(days=1)].values

In [0]:
# datos_cong_balanceado[['x','y','bearing_t','speed_t']].tail(288)

In [0]:
# Identificamos las columnas a completar usando los datos del día anterior
columns_to_fill = ['x', 'y', 'bearing_t', 'speed_t']

# Rellenamos las columnas específicas con los valores del día anterior
for col in columns_to_fill:
    future_dates_pronostico[col] = df_and_future[col].loc[future - pd.Timedelta(days=1)].values

# Las otras columnas se quedan con NaN o el valor que tenías en el DataFrame de predicción
future_dates_pronostico.head()

Predict the future

In [0]:
FEATURES = ['x','y','bearing_t','speed_t','lag1', 'lag2', 'lag3','hora','minuto']

In [0]:
FEATURES

In [0]:
column_pred_congestion = xgb_model.predict_proba(future_dates_pronostico[FEATURES])
column_pred_congestion = column_pred_congestion[:, 1]
future_dates_pronostico['congestion_pred']=column_pred_congestion

In [0]:
future_dates_pronostico.head()

In [0]:
future_dates_pronostico['congestion_pred'].plot(figsize=(10,5),
                                                ms=1,lw=1, title='Future Prediction Congestion 24-4-2024')

Saving Model ML Time Series XGB for Later

In [0]:
xgb_model.save_model('model_xgb_timeseries.json')

Cargar el Modelo XGB Time Series para predecir

In [0]:
import xgboost as xgb
model_xgb_new = xgb.XGBClassifier()
model_xgb_new.load_model('model_xgb_timeseries.json')

column_pred_congestion2 = model_xgb_new.predict_proba(future_dates_pronostico[FEATURES])
column_pred_congestion2 = column_pred_congestion2[:, 1]
future_dates_pronostico['congestion_pred_2']=column_pred_congestion2

future_dates_pronostico['congestion_pred_2'].plot(figsize=(10,5),
                                                ms=1,lw=1, title='Future Prediction Congestion 24-4-2024')

In [0]:
import seaborn as sns
mask=future_dates_pronostico['x']>0 
data=future_dates_pronostico[mask]
sns.scatterplot(data['x'],data['y'], hue=data['congestion_pred_2'])

#### Algoritmo LSTM para Time Series

Cargamos los datos

In [0]:
#1. Leemos los datos de PROCEESED la tabla Delta 
df_delta2 = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/data_congestion_serietemp_balanceada")
datos_cong_balanceado = df_delta2.toPandas()
datos_cong_balanceado.head()

Ajuste de los datos para problemas de Time Series

In [0]:
# #2. Ajuste de los datos a Series Temporales
# #2.1 Se establece la columna 'Time' como el índice del DataFrame "datos"
datos_cong_balanceado = datos_cong_balanceado.set_index('instant_date_t')

# #2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos_cong_balanceado.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos_cong_balanceado.index.to_series().diff().dt.total_seconds()

# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
# diferencias_frecuencias = df_time_diffs.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
# print(diferencias_frecuencias)

#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
datos_cong_balanceado = datos_cong_balanceado[df_time_diffs != 0]

#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
datos_cong_balanceado = datos_cong_balanceado.asfreq(freq='6s', method='bfill')

# Filtramos los valores de x, y, z que son 0s
mask = (datos_cong_balanceado['x'] == 0) & (datos_cong_balanceado['y'] == 0) & (datos_cong_balanceado['z'] == 0)
datos_cong_balanceado = datos_cong_balanceado[~mask]

datos_cong_balanceado.head()

Definir si sera Modelo Univariado o Multivariado

In [0]:
#datos_df = datos_cong_balanceado['congestion']  # Modelo Univariado
datos_df = datos_cong_balanceado[['x','speed_t','congestion']]  # Modelo Multivariado

#### Escalar/Normalizar los datos 
Es requerido para garantizar que todas las caracteristicas se encuentren en el mismo rango
de valores, lo que facilita el entrenamiento del Modelo y las predicciones

In [0]:
import numpy as np

def escalar_por_variables_especificas(dataframe, variables_a_escalar, medias, desviaciones_estandar):
  """
  Escala solo las variables especificadas en un dataframe de forma independiente.

  Args:
    dataframe: El dataframe que se va a escalar.
    variables_a_escalar: Una lista que contiene los nombres de las variables que se van a escalar.
    medias: Un diccionario que contiene la media de cada variable.
    desviaciones_estandar: Un diccionario que contiene la desviación estándar de cada variable.

  Returns:
    El dataframe escalado.
  """
  dataframe_escalado = dataframe.copy()
  for columna in dataframe.columns:
    if columna in variables_a_escalar:
      dataframe_escalado[columna] = (dataframe[columna] - medias[columna]) / desviaciones_estandar[columna]
  return dataframe_escalado

In [0]:

# Definir la lista de variables a escalar
variables_a_escalar = ["x", "speed_t"]  # Reemplazar con los nombres reales de las columnas

# Calcular medias y desviaciones estándar para todas las variables
medias = datos_df.mean()
desviaciones_estandar = datos_df.std()

# Escalar solo las variables especificadas en el dataframe train
df_escalado = escalar_por_variables_especificas(datos_df, variables_a_escalar, medias, desviaciones_estandar)

In [0]:
df_escalado.head()

Metodo de Desviacion estandar y MEDIA

In [0]:
# train_mean = tr.mean() # Guardar este valor para usarlo cuando despliegues el Modelo en Produccion
# train_std = tr.std()  # Guardar este valor para usarlo cuando despliegues el Modelo en Produccion 

# tr = (tr - train_mean) / train_std
# vl = (vl - train_mean) / train_std
# ts = (ts - train_mean) / train_std


# INVERSION DE ESCALAMIENTO (CUANDO DESPLIEGUES EN PRODUCCION)
# def invertir_escalamiento(predicciones, train_mean, train_std):
#     return predicciones * train_std + train_mean

# # Suponiendo que 'predicciones' son las predicciones de tu modelo
# predicciones_invertidas = invertir_escalamiento(predicciones, train_mean, train_std)

Metodo de Min Max

In [0]:
# ESCALAMIENTO
# train_min = tr.min()   # Guardar este valor para usarlo cuando despliegues el Modelo en Produccion
# train_max = tr.max()   # Guardar este valor para usarlo cuando despliegues el Modelo en Produccion

# tr = (tr - train_min) / (train_max - train_min)
# vl = (vl - train_min) / (train_max - train_min)
# ts = (ts - train_min) / (train_max - train_min)

# INVERSION DE ESCALAMIENTO (CUANDO DESPLIEGUES EN PRODUCCION)
# def invertir_minmax_escalamiento(predicciones, train_min, train_max):
#     return predicciones * (train_max - train_min) + train_min

# # Suponiendo que 'predicciones' son las predicciones de tu modelo
# predicciones_invertidas = invertir_minmax_escalamiento(predicciones, train_min, train_max)

5.1 Particion del Set en entrenamiento, validacion y prueba
- En el caso de series temporales se debe garantizar que se generan las particiones sin mezclar aleaotariamente

Recordemos

. El set entrenamiento se usa para encontrar los parametros 

. El set validacion para verificar que no haya under/over fitting

. El set de prueba para poner a prueba el mejor modelo encontrado

In [0]:
#Funcion para generar las particiones, siguiendo la secuencia de las series temporales
def train_val_test_split(serie, tr_size =0.8, vl_size=0.1, ts_size=0.1):
    # Definir nunmero de datos en cada subserie
    N = serie.shape[0]   #Serie seria el df, con las columnas que intervendran en el Modelo
    Ntrain = int(tr_size*N)  # Numero de datos de entrenamiento
    Nval = int(vl_size*N)
    Ntest = int(ts_size*N)

    #Realizar la particion
    train = serie[0:Ntrain]
    val = serie[Ntrain:Nval+Ntrain]
    test = serie[Nval+Ntrain:]

    return train, val, test


# Prueba de la funcion 
tr, vl, ts  = train_val_test_split(datos_df)

print(f'Shape del Set de Entrenamiento, {tr.shape}')
print(f'Shape del Set de Validacion, {vl.shape}')
print(f'Shape del Set de Test, {ts.shape}')

In [0]:
tr.head()

In [0]:
#datos_cong_balanceado.mean()

In [0]:

import matplotlib.pyplot as plt
#Dibujemos los subsets
fig, ax  = plt.subplots(figsize  = (15,5))
ax.plot(tr, label = 'Train')
ax.plot(vl, label = 'Validation')
ax.plot(ts, label = 'Test')
plt.legend()

Grafico de Cada Serie Temporal (Analisis Multivariado)

In [0]:
covar  = 1# Indice de la covariable (columna del dataset) a graficar
col  = datos_df.columns[covar]

# Dibujar los sets de entrenamiento, validacion, prueba
fig, ax = plt.subplots(figsize=(16,5))
ax.plot(tr[col], label='Train')
ax.plot(vl[col], label='Validation')
ax.plot(ts[col], label='Test')
ax.set_title(f'Variable, {col}')
plt.legend()

###5.2 Generación del dataset supervisado (entrada y salida del modelo)

Debemos ajustar nuestro set de datos de acuerdo a lo especificado en la documentacion de LSTM

In [0]:
from IPython.display import Image

# Asegúrate de que la ruta de la imagen sea correcta
img_path = 'entradas_salidas_LSTM.png'

# Muestra la imagen
Image(filename=img_path)

In [0]:
from IPython.display import Image

# Asegúrate de que la ruta de la imagen sea correcta
img_path = 'lstm_multivariado.png'

# Muestra la imagen
Image(filename=img_path)

Así que en este caso cada dato de entrenamiento será:

- Un arreglo de 24 (horas) x 1 (feature) correspondiente a la entrada
- Un arreglo de 1 (hora) x 1 (feature) correspondiente a la hora 25 (a predecir):

In [0]:
from IPython.display import Image

# Asegúrate de que la ruta de la imagen sea correcta
img_path = 'dataset_supervisado.png'

# Muestra la imagen
Image(filename=img_path)

Creamos la funcion que pondremos reutilizar mas adelante para implementar modelos mas complejos

In [0]:
def crear_dataset_supervisado(array, input_length, output_length): 
    ''' Permite crear un dataset con las entradas (X) y salidas (Y)
    requeridas por las Red LSTM.

    Parametros :
    - array : arreglo numpy de tamano N * Features (N: cantidad de datos, Features: Cantidad de features)
    -  input_length : Instantes de tiempo consecutivos de la(s) serie(s) de tiempo usados para alimentar al modelo (bloques de bash para ingresar ala LSTM)
    -  output_length : instantes de tiempo a pronosticar (salida del modelo)
    '''
    #Inizializacion
    X, Y = [], []   #Listados que contendran los datos de entrada y salida del modelo (Lotes o Batches)
    shape = array.shape
    if len(shape) == 1:  #Si tenemos solo una variable(serie) UNIVARIADO
        fils, cols = array.shape[0], 1   #UNIVARIADO (Por ello solo 1 columna)
        array = array.reshape(fils,cols)
    else: #MULTIVARIDAO
        fils, cols = array.shape

    #Generar los arreglos
    for i in range(fils-input_length-output_length):
        X.append(array[i:i+INPUT_LENGTH,0:cols])
        Y.append(array[i+input_length:i+input_length+output_length,-1].reshape(output_length,1))
    
    #Convertir listas a arreglos Numpy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

In [0]:
import numpy as np
#Crear los datasets de entrenamiento, validacion, prueba
INPUT_LENGTH = 2 # Numero de Pasos en el pasado
OUTPUT_LENGTH = 1  # Modelo Uni Step (1 en este casoi indica 1 fila adelante), Multistep mas de 1 

x_tr, y_tr = crear_dataset_supervisado(tr.values, INPUT_LENGTH, OUTPUT_LENGTH)
x_vl, y_vl = crear_dataset_supervisado(vl.values, INPUT_LENGTH, OUTPUT_LENGTH)
x_ts, y_ts = crear_dataset_supervisado(ts.values, INPUT_LENGTH, OUTPUT_LENGTH)

#Imprimimos la informacion en pantalla
print('Tamanos entrada (BATCHES x INPUT_LENGTH x FEATURES) y de salida (BACTHES x OUTPUT_LENGTH x FEATURES)')

print(f'Set de entenamiento x_tr : {x_tr.shape}, y_tr : {y_tr.shape}')
print(f'Set de validacion x_vl : {x_vl.shape}, y_vl : {y_vl.shape}')
print(f'Set de entenamiento x_ts : {x_ts.shape}, y_ts : {y_ts.shape}')

In [0]:
tr.head(10)

In [0]:
x_tr[:5]

In [0]:
y_tr

#### Construccion y Entrenamiento del Modelo
- Entradas : arreglos x (batches x input_length x features = batches x 24 x 1)
- Salidas : arreglos y (batches x output_length x features = batches x 1 x 1)

Usaremos la RSME para medir el performance del Modelo

In [0]:
x_tr.shape[1], x_tr.shape[2]

In [0]:
# Construccion del Modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf

# Fijar Valores parametros , para asegurar la reproducibilidad de los datos inicializados
tf.random.set_seed(123) # Semilla(inicializacion de los parametros o pesos de misma manera)
tf.config.experimental.enable_op_determinism() # Cada vez se ejecute de misma manera

# El Modelo
N_UNITS = 150 # Tamaño del estado oculto de la celda de Memoria de LSTM
INPUT_SHAPE = (x_tr.shape[1], x_tr.shape[2]) #50 pasos atras x n features (feature)

modelo = Sequential()
modelo.add(LSTM(N_UNITS, input_shape=INPUT_SHAPE))
modelo.add(Dense(OUTPUT_LENGTH, activation='sigmoid')) # linear: problema regresion, sigmoid:clasificacion binaria, softmax:multiclase

#RSME: Para problemas de regresion, para tener errores en las mismas unidades de la variable target
# def root_mean_squared_error(y_true,y_pred):
#     rmse=tf.math.sqtr(tf.math.reduce_mean(tf.square(y_pred-y_true)))
#     return rmse

# Optimizador para problema de regresion
# optimizador = RMSprop(learning_rate=0.05)

#Compilation


# Compilar el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','AUC'])

# Entrenamiento del Modelo
EPOCHS = 50 # Numero de epocas de entrenamiento
BATCH_SIZE = 256 # Numero de lotes que van ingresando al Modelo(Algoritmo)
historia = modelo.fit(
    x = x_tr,
    y = y_tr,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = (x_vl, y_vl),
    verbose=2  #Imprime en pantalla con va el entrenamiento
)

#### Graficar curvas de entrenamiento y validacion, para verificar que no existe overfitting

In [0]:
plt.plot(historia.history['loss'],label='Loss Train')
plt.plot(historia.history['val_loss'],label='Loss Validation')
plt.xlabel('Iteracion')
plt.ylabel('Loss')
plt.legend()

In [0]:
plt.plot(historia.history['accuracy'],label='Accuracy Train')
plt.plot(historia.history['val_accuracy'],label='Accuracy Validation')
plt.xlabel('Iteracion')
plt.ylabel('Accuracy')
plt.legend()

#### 7.Performance del Modelo
Verificamos el rendimiento para el set de test , y comparamos con el de 
entrenamiento y validacion.
Este rendimiento es simplemente el LOSS obtenido con cada subset

In [0]:
# Calculo de Loss Binary para train, val, test
loss_tr = modelo.evaluate(x=x_tr, y=y_tr, verbose=0) #verbose=0 No se imprime el progreso
loss_vl = modelo.evaluate(x=x_vl, y=y_vl, verbose=0) #No se imprime el progreso
loss_ts = modelo.evaluate(x=x_ts, y=y_ts, verbose=0) #No se imprime el progreso

# Los valores de pérdida son los primeros elementos de las listas devueltas por evaluate()
loss_tr_value = loss_tr[0]
loss_vl_value = loss_vl[0]
loss_ts_value = loss_ts[0]

# Imprimir resultados en pantalla
print('Comparativo rendimiento:')
print(f'Loss train {loss_tr_value:.3f}')
print(f'Loss val {loss_vl_value:.3f}')
print(f'Loss test {loss_ts_value:.3f}')

#### 8. Predicciones en el Datos de Test con el Modelo Entrenado

In [0]:
def predecir(x, model):
    '''Genera la prediccion OUTPUT_LENGTH de tiempo a futuro con el modelo entrenado
    Entrada: 
    X: batch de datos para ingresar al modelo
    model: Red LSTM

    Salida:
    y_pred:prediccion en la escala original (tamano de Bacthes)
    '''
    y_pred = modelo.predict(x,verbose=0)

    # Llevar la prediccion a la escala original
    # train_std=0
    # train_mean=0
    # y_pred_inversa = y_pred * train_std + train_mean

    return y_pred.flatten()

In [0]:
# Calcular prediciones sobre el datos de TEST
y_ts_pred = predecir(x_ts, modelo)


In [0]:
x_ts[0]

In [0]:
len(y_ts_pred)

Opcional : En caso sea problema de Regresion , se puede tambien graficar la grafica de errores del Modelo, osea en cuantas unidades se equivoco el #modelo al momento de predecir

In [0]:
## En caso sea problema de Regresion , se puede tambien graficar la grafica de errores del Modelo, osea en cuantas unidades se equivoco el #modelo al momento de predecir
N = len(y_ts)
ndato=np.linspace(1,N,N)

#Calculo de errores simples
errores = y_ts.flatten()-y_ts_pred
plt.plot(errores)

### Pronostico al Futuro usando la Red LSTM entrenada

In [0]:
datos_cong_balanceado.index.max()

In [0]:
#future=pd.date_range(datos_cong_balanceado.index.max() + pd.Timedelta(minutes=5), datos_cong_balanceado.index.max() + pd.Timedelta(days=1) , #freq='5T')
datos_cong_balanceado["congestion"][datos_cong_balanceado.index.max() - pd.Timedelta(days=1) :datos_cong_balanceado.index.max()].head()

In [0]:
# Seleccionamos los datos del Ultimo dia de datos, para hacer la prediccion al futuro usando el Modelo Entrenado LSTM(fue entrenado con 50 #pasos atras)
ultimosDias = datos_cong_balanceado["congestion"][datos_cong_balanceado.index.max() - pd.Timedelta(days=1) :datos_cong_balanceado.index.max()]

In [0]:
ultimosDias.values

In [0]:
#Filtrar solo la ultima fila del array, lo que representaria los ultimas fechas del dataset
reframed.values[-1]

In [0]:
#De este conjunto “ultimosDias” tomamos sólo la última fila, pues es la que correspondería a la última semana de noviembre y la dejamos en el #formato correcto para la red neuronal con reshape:
values = reframed.values
x_test = reframed.values[-1]
x_test = x_test.reshape((x_test.shape[0], 1))
x_test

In [0]:
x_ts[0].shape

In [0]:
x_test.shape

Ahora crearemos una función para ir “rellenando” el desplazamiento que hacemos por cada predicción. Esto es porque queremos predecir los pasos adelante (en este caso 1 dia siguiente).

In [0]:
import numpy as np

# Función para actualizar la secuencia con el nuevo valor predicho
def actualizarSecuencia(x_test, nuevo_valor):
    # Desplazar los valores a la izquierda
    for i in range(x_test.shape[0] - 1):
        x_test[i] = x_test[i + 1]
    # Añadir el nuevo valor predicho al final de la secuencia
    x_test[-1] = nuevo_valor
    return x_test

def agregarNuevoValor(x_test,nuevoValor):
    for i in range(x_test.shape[2]-1):
        x_test[0][0][i] = x_test[0][0][i+1]
    x_test[0][0][x_test.shape[2]-1]=nuevoValor
    return x_test

In [0]:
x_test

In [0]:
modelo.predict(x_test)[0]

In [0]:

# Número de pasos hacia el futuro que quieres predecir
n_steps = 150

# Lista para almacenar los resultados de las predicciones
resultados = []

# Bucle para hacer las predicciones
for _ in range(n_steps):
    # Hacer una predicción sobre la secuencia actual de x_test
    prediccion = modelo.predict(x_test)
    # Almacenar la predicción
    resultados.append(prediccion[0])
    # Actualizar la secuencia de entrada con el nuevo valor predicho
    x_test = actualizarSecuencia(x_test, prediccion[0])

# Mostrar los resultados de las predicciones
print("Predicciones futuras:", resultados)

In [0]:
adimen = [x for x in resultados]

In [0]:
adimen

In [0]:
prediccion_24abril = pd.DataFrame(adimen)
prediccion_24abril.columns = ['pronostico']
prediccion_24abril.plot()

##  Time Series how Problem de Aprendizaje supervisado 

1. Ejemplo de explicacion como podriamos pasar de problema de serie temporal a aprendizaje supervisado

In [0]:
from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
df['t-1'] = df['t'].shift(1)  # 1 un paso atras, =1 un paso adelante
df['t-2'] = df['t'].shift(2)
print(df)

In [0]:
values = [x for x in range(10)]
data = series_to_supervised(values, 3)
print(data)

In [0]:
raw = DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
values = raw.values
values
#data = series_to_supervised(values)
#print(data)

In [0]:
data = series_to_supervised(values)
print(data)

Cargamos los datos a Analizar y pasar de Time Series a Supervised Learning

In [0]:
#1. Leemos los datos de PROCEESED la tabla Delta 
# df_delta2 = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/tablacaracteristicas_congestion_tabladelta_v3")

df_delta2 = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/data_congestion_serietemp_balanceada")

datos_cong_balanceado = df_delta2.toPandas()
datos_cong_balanceado.head()

Ajuste de los datos para problemas de Time Series(Establecer la Frecuencia )

In [0]:
# #2. Ajuste de los datos a Series Temporales
# #2.1 Se establece la columna 'Time' como el índice del DataFrame "datos"
datos_cong_balanceado = datos_cong_balanceado.set_index('instant_date_t')

# #2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos_cong_balanceado.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos_cong_balanceado.index.to_series().diff().dt.total_seconds()

# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
# diferencias_frecuencias = df_time_diffs.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
# print(diferencias_frecuencias)

#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
datos_cong_balanceado = datos_cong_balanceado[df_time_diffs != 0]

#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
#datos_cong_balanceado = datos_cong_balanceado.asfreq(freq='5T', method='bfill')
datos_cong_balanceado = datos_cong_balanceado.asfreq(freq='6s', method='bfill')

# Filtramos los valores de x, y, z que son 0s
mask = (datos_cong_balanceado['x'] == 0) & (datos_cong_balanceado['y'] == 0) & (datos_cong_balanceado['z'] == 0)
datos_cong_balanceado = datos_cong_balanceado[~mask]

datos_cong_balanceado.head()

In [0]:
datos_cong_balanceado.shape

In [0]:
datos_cong_balanceado['congestion'].value_counts()

In [0]:
#Va
datos_cong_balanceado['congestion'].value_counts()

Eleccion de si es Modelo Univariado o Multivariado (Poner la columna a Predecir en la ultima columna )

In [0]:
#datos_df = datos_cong_balanceado['congestion']  # Modelo Univariado


#En Multivariado, poner la variable a predecir, en la columna final
# c_variables = ['id_equipo','id_worker','id_path','n_sat','isload_t','marcha_t','precisiongps_t','x','y','z','direccion_t'
# ,'speed_t','pitch_t','roll_t','segment_angle_t','tonelaje_t','fuel_rate_t','combustibleint_t'
# ,'LCKUP_SLIP','BRK/AIR_PRES','RTF_LTF_BRKTEMP','RTR_LTR_BRKTEMP','RT_F_BRK_TEMP','RT_R_BRK_TEMP','LT_R_BRK_TEMP','SERV_BRK_STAT','Tire_Press_N4','Tire_Press_N6','Hourmeter_MSPU','Direction','congestion']

c_variables = ['x','speed_t','congestion']


datos_df = datos_cong_balanceado[c_variables]  # Modelo Multivariado
datos_df.head()

Escalar/Normalizar los datos 
- Es requerido para garantizar que todas las caracteristicas se encuentren en el mismo rango
de valores, lo que facilita el entrenamiento del Modelo y las predicciones

In [0]:
import numpy as np

def escalar_por_variables_especificas(dataframe, variables_a_escalar, medias, desviaciones_estandar):
  """
  Escala solo las variables especificadas en un dataframe de forma independiente.

  Args:
    dataframe: El dataframe que se va a escalar.
    variables_a_escalar: Una lista que contiene los nombres de las variables que se van a escalar.
    medias: Un diccionario que contiene la media de cada variable.
    desviaciones_estandar: Un diccionario que contiene la desviación estándar de cada variable.

  Returns:
    El dataframe escalado.
  """
  dataframe_escalado = dataframe.copy()
  for columna in dataframe.columns:
    if columna in variables_a_escalar:
      dataframe_escalado[columna] = (dataframe[columna] - medias[columna]) / desviaciones_estandar[columna]
  return dataframe_escalado

In [0]:

# Definir la lista de variables a escalar
variables_a_escalar = ["x", "speed_t"]  # Reemplazar con los nombres reales de las columnas

# Calcular medias y desviaciones estándar para todas las variables
medias = datos_df.mean()
desviaciones_estandar = datos_df.std()

# Escalar solo las variables especificadas en el dataframe train
df_escalado = escalar_por_variables_especificas(datos_df, variables_a_escalar, medias, desviaciones_estandar)

Generacion del DataSet Supervisado
- Función de Python llamada series_to_supervised()

In [0]:
# Función de Python llamada series_to_supervised() que toma una serie temporal univariada o multivariada y la encuadra como un #conjunto de datos de aprendizaje supervisado.
import pandas as pd
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
    data: Secuencia de observaciones como una lista o matriz NumPy 2D.
    n_in : número de observaciones de retraso como entrada ( X ). Los valores pueden estar entre [1..len(data)] Opcional. El valor predeterminado es 1.
    n_out : Número de observaciones como salida ( y ). Los valores pueden estar entre [0..len(data)-1]. Opcional. El valor predeterminado es 1.
    dropnan : valor booleano para eliminar o no filas con valores NaN. Opcional. El valor predeterminado es Verdadero.

    Returns:
    Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))    # Aplicas los pasos hacia atras 
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))   # Aplicas los pasos hacia adelante
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)    #Eliminamos las filas Nulas de nuestro df generado(Porque no sirven en la serie temporal)
    return agg

In [0]:
# Crear los datasets de entrenamiento, prueba y validación y verificar sus tamaños
numero_pasos_atras = 1  # Hiperparámetro
numero_pasos_futuro = 1   # Modelo uni-step igual a 1 paso,   MUltivariado seria mayor a 1 paso al futuro

# tr_s = series_to_supervised(tr.values, numero_pasos_atras , numero_pasos_futuro)
# vl_s = series_to_supervised(vl.values, numero_pasos_atras, numero_pasos_futuro)
# ts_s = series_to_supervised(ts.values, numero_pasos_atras, numero_pasos_futuro)

datos_df_s = series_to_supervised(datos_df.values, numero_pasos_atras, numero_pasos_futuro)

# Asignar nombres originales a las columnas
original_columns = datos_df.columns

new_columns = []
for i in range(numero_pasos_atras, 0, -1):
    new_columns += [f'{col}(t-{i})' for col in original_columns]
for i in range(0, numero_pasos_futuro):
    if i == 0:
        new_columns += [f'{col}(t)' for col in original_columns]
    else:
        new_columns += [f'{col}(t+{i})' for col in original_columns]

datos_df_s.columns = new_columns
datos_df_s.head()

In [0]:
datos_df_s.head()

Solo nos quedamos en la ultima columna con la columna a predecir (Las demas que estan en el tiempo T se eliminan)

In [0]:
# Solo nos quedariamos con la variable congestion(t)  como target
columns_to_drop = ['x(t)', 'speed_t(t)']
datos_df_s.drop(columns=columns_to_drop, inplace=True)

In [0]:
datos_df_s.head()

In [0]:
datos_df_s.shape

Separamos los datos en Train, Validation, Test

In [0]:
#Funcion para generar las particiones, siguiendo la secuencia de las series temporales
def train_val_test_split(serie, tr_size =0.8, vl_size=0.1, ts_size=0.1):
    # Definir nunmero de datos en cada subserie
    N = serie.shape[0]   #Serie seria el df, con las columnas que intervendran en el Modelo
    Ntrain = int(tr_size*N)  # Numero de datos de entrenamiento
    Nval = int(vl_size*N)
    Ntest = int(ts_size*N)

    #Realizar la particion
    train = serie[0:Ntrain]
    val = serie[Ntrain:Nval+Ntrain]
    test = serie[Nval+Ntrain:]

    return train, val, test


# Prueba de la funcion 
tr, vl, ts  = train_val_test_split(datos_df_s)

print(f'Shape del Set de Entrenamiento, {tr.shape}')
print(f'Shape del Set de Validacion, {vl.shape}')
print(f'Shape del Set de Test, {ts.shape}')

Separamos las X(Variables predictoras) e Y(variable target), para luego poner en X_train, etc

In [0]:
def separar_X_y(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return X, y

# Separar características y objetivo para cada conjunto
X_train, y_train = separar_X_y(tr, 'congestion(t)')
X_val, y_val = separar_X_y(vl, 'congestion(t)')
X_test, y_test = separar_X_y(ts, 'congestion(t)')

In [0]:
datos_df.head(10)

In [0]:
X_train.head()

In [0]:
X_train.shape

La red LSTM espera que los datos de entrada (X) se proporcionen con una estructura de matriz específica en forma de: [muestras, pasos de tiempo, características]

In [0]:
import numpy as np
n_features = X_train.shape[1]  # número de características predictoras
# Función para reshape de X para LSTM
def reshape_X_for_lstm(X, numero_pasos_atras, n_features):
    n_samples = X.shape[0] - numero_pasos_atras - numero_pasos_futuro + 1
    X_lstm = np.zeros((n_samples, numero_pasos_atras, n_features))
    for i in range(n_samples):
        X_lstm[i] = X.iloc[i:i+numero_pasos_atras, :].values
    return X_lstm

# Aplicar reshape para XS( X_train, X_val, X_test) para LSTM
X_train_lstm = reshape_X_for_lstm(X_train, numero_pasos_atras, n_features)
X_val_lstm = reshape_X_for_lstm(X_val, numero_pasos_atras, n_features)
X_test_lstm = reshape_X_for_lstm(X_test, numero_pasos_atras, n_features)

# Función para reshape para Ys () para LSTM
def reshape_y_for_lstm(y, numero_pasos_futuro):
    return y.iloc[numero_pasos_atras + numero_pasos_futuro - 1:].values.reshape(-1, numero_pasos_futuro, 1)

# Aplicar reshape para y_train, y_val, y_test
y_train_lstm = reshape_y_for_lstm(y_train, numero_pasos_futuro)
y_val_lstm = reshape_y_for_lstm(y_val, numero_pasos_futuro)
y_test_lstm = reshape_y_for_lstm(y_test, numero_pasos_futuro)

# Verificar las formas después del reshape para LSTM
print("Shape de X_train_lstm:", X_train_lstm.shape)
print("Shape de y_train_lstm:", y_train_lstm.shape)
print("Shape de X_val_lstm:", X_val_lstm.shape)
print("Shape de y_val_lstm:", y_val_lstm.shape)
print("Shape de X_test_lstm:", X_test_lstm.shape)
print("Shape de y_test_lstm:", y_test_lstm.shape)

In [0]:
X_train_lstm[:5]

In [0]:
X_train_lstm[2]

#### Metodos para pasar a formato de input para Redes Neuronales LSTM
Ambos métodos pueden ser válidos dependiendo del contexto y la estructura de los datos.

(282653, 2, 7), (282653, 1, 1) para entrenamiento:

Aquí, cada muestra tiene 2 pasos de tiempo y 7 características para las variables predictoras, y un solo valor de salida. Esto es adecuado si estás modelando datos donde tienes múltiples pasos de tiempo y varias características.


(282653, 14), (282653,) para entrenamiento:

En este caso, todas las características se aplanan en una sola dimensión, lo que significa que tratas todas las características como una secuencia única de datos por muestra. Esto puede ser útil en algunos casos donde no necesitas modelar explícitamente múltiples pasos de tiempo, pero simplemente quieres predecir un solo valor en el siguiente paso de tiempo.

In [0]:
X_train_lstm.shape

In [0]:
# Construccion del Modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf

# Fijar Valores parametros , para asegurar la reproducibilidad de los datos inicializados
tf.random.set_seed(123) # Semilla(inicializacion de los parametros o pesos de misma manera)
tf.config.experimental.enable_op_determinism() # Cada vez se ejecute de misma manera

# El Modelo
N_UNITS = 50 # Tamaño del estado oculto de la celda de Memoria de LSTM
INPUT_SHAPE = (X_train_lstm.shape[1], X_train_lstm.shape[2]) #pasos atras x n features (feature)

modelo = Sequential()
modelo.add(LSTM(N_UNITS, input_shape=INPUT_SHAPE))
modelo.add(Dense(numero_pasos_futuro, activation='sigmoid')) # linear: problema regresion, sigmoid:clasificacion binaria, softmax:multiclase

#RSME: Para problemas de regresion, para tener errores en las mismas unidades de la variable target
# def root_mean_squared_error(y_true,y_pred):
#     rmse=tf.math.sqtr(tf.math.reduce_mean(tf.square(y_pred-y_true)))
#     return rmse

# Optimizador para problema de regresion
# optimizador = RMSprop(learning_rate=0.05)

#Compilation


# Compilar el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','AUC'])

# Entrenamiento del Modelo
EPOCHS = 50 # Numero de epocas de entrenamiento
BATCH_SIZE = 256 # Numero de lotes que van ingresando al Modelo(Algoritmo)
historia = modelo.fit(
    x = X_train_lstm,
    y = y_train_lstm,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = (X_val_lstm, y_val_lstm),
    verbose=2  #Imprime en pantalla con va el entrenamiento
)

In [0]:
import pickle